In [30]:
import pyspark, os, sys
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import types


spark=SparkSession.builder.getOrCreate()

df=spark.read.format("csv").option("header", True).option("inferSchema", True).load("sample1.csv")
#df.show()
#df.groupBy("City").count().show()

df.write.option("header", True).partitionBy("City").mode("overwrite").csv("/pyspark_practice/partitionBy_output_files") #Here we are partitioning based on one column which is 'City'
df2=spark.read.format("csv").option("header", True).option("inferSchema", True).load("/pyspark_practice/partitionBy_output_files/")
#df2.show()
df2.rdd.getNumPartitions()  #3 partitions will be created while writing csv file in putput location beacuse we are grouping by 'city' and there are 3 cities in input csv file.

#we are performing partitions based on 2 columns 'City' and 'Zipcode'
df.write.option("header", True).partitionBy("City","Zipcode").mode("overwrite").csv("/pyspark_practice/partitionBy_output_files_multiple_columns/")
df3=spark.read.format("csv").option("header", True).option("inferSchema", True).load("/pyspark_practice/partitionBy_output_files_multiple_columns/")
#df3.show()
df3.rdd.getNumPartitions()


df.write.option("header", True).option("maxRecordsPerFile",1).partitionBy("City").mode("overwrite").csv("/pyspark_practice/max_records_per_partition/")

df4=spark.read.format("csv").option("header", True).option("inferSchema", True).load("/pyspark_practice/max_records_per_partition/")
#df4.show()
df4.rdd.getNumPartitions()

3

In [48]:
#to check no. of records per partition
from pyspark.sql.functions import spark_partition_id  #this library is used to assign id to each record within the partition 
spark=SparkSession.builder.getOrCreate()

df=spark.read.format("csv").option("header", True).option("inferSchema", True).load("sample1.csv")
#df.show()

df.withColumn("partitionId", spark_partition_id()).groupBy('partitionId').count()#.show()

df.rdd.getNumPartitions()  #Here it will show 1 as output as we have not done any adjustment in partition or repartition
df_new=df.select(df.Id, df.City, df.Zipcode).repartition(5)
df_new.show()
df_new.rdd.getNumPartitions()  #Here we will get 5 partitions as we have done repartition above

df_new.withColumn("partitionId", spark_partition_id()).groupBy("partitionId").count().show() 

+---+-------+-------+
| Id|   City|Zipcode|
+---+-------+-------+
|  3| Mumbai| 400000|
|  4|Newyork|  10000|
|  2| Nashik|  41111|
|  1| Mumbai| 400000|
|  5|Newyork|  10001|
+---+-------+-------+

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|    1|
|          1|    1|
|          2|    1|
|          3|    1|
|          4|    1|
+-----------+-----+

